<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Get profile posts stats
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_profile_posts_stats.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Get+profile+posts+stats:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #linkedin #profile #post #stats #naas_drivers #content #automation #csv

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook provides a way to track and analyze the performance of posts on your LinkedIn profile.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin
import pandas as pd
from datetime import datetime
import naas

### Setup LinkedIn
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# LinkedIn cookies
LI_AT = (
    naas.secret.get("LI_AT") or "ENTER_YOUR_COOKIE_HERE"
)  # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = (
    naas.secret.get("JSESSIONID") or "ENTER_YOUR_JSESSIONID_HERE"
)  # EXAMPLE : "ajax:8379907400220387585"

# LinkedIn profile url
PROFILE_URL = "ENTER_YOUR_LINKEDIN_PROFILE_HERE"  # EXAMPLE "https://www.linkedin.com/in/myprofile/"

### Setup Outputs
Create CSV to store your posts stats.<br>
PS: This CSV could be used in others LinkedIn templates.

In [ ]:
# Custom path of your CSV with the profile URL
profile_id = PROFILE_URL.split("https://www.linkedin.com/in/")[-1].split("/")[0]
csv_output = f"LINKEDIN_PROFILE_POSTS_{profile_id}.csv"

### Setup Naas scheduler
Schedule your notebook with the naas scheduler feature

In [ ]:
# the default settings below will make the notebook run everyday at 8:00
# for information on changing this setting, please check https://crontab.guru/ for information on the required CRON syntax
naas.scheduler.add(cron="0 8 * * *")

# to de-schedule this notebook, simply run the following command:
# naas.scheduler.delete()

## Model

### Get your posts from CSV
All your posts will be stored in CSV.

In [ ]:
def read_csv(file_path):
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError as e:
        # Empty dataframe returned
        return pd.DataFrame()
    return df


df_posts = read_csv(csv_output)
print("✅ Posts fetched:", len(df_posts))
df_posts.head(1)

### Update last posts
This function will only update the last 5 posts from LinkedIn API.<br>
To change this parameters you can set another number to the parameter "no_posts" in the update_posts() function.

PS: On the first execution all posts will be retrieved.

In [ ]:
def update_posts(
    df_posts, profile_url, key="POST_URL", no_posts=5, min_updated_time=60
):
    # Init output
    df = pd.DataFrame()
    df_new = pd.DataFrame()

    # Init df posts is empty then return entire database
    if len(df_posts) > 0:
        if "DATE_EXTRACT" in df_posts.columns:
            last_update_date = df_posts["DATE_EXTRACT"].max()
            time_last_update = datetime.now() - datetime.strptime(
                last_update_date, "%Y-%m-%d %H:%M:%S"
            )
            minute_last_update = time_last_update.total_seconds() / 60
            if minute_last_update > min_updated_time:
                # If df posts not empty get the last X posts (new and already existing)
                df_new = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(
                    profile_url, limit=no_posts
                )
            else:
                print(
                    f"🛑 Nothing to update. Last update done {int(minute_last_update)} minutes ago."
                )
    else:
        df_new = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(
            profile_url, limit=-1
        )

    # Concat, save database in CSV and dependency in production
    df = pd.concat([df_new, df_posts]).drop_duplicates(key, keep="first")

    # Return all posts
    print(f"✅ Updated posts fetched:", len(df))
    return df.reset_index(drop=True)


df_update = update_posts(df_posts, PROFILE_URL)
df_update.head(1)

## Output

### Save dataframe in CSV and send to production

In [ ]:
# Save dataframe in CSV
df_update.to_csv(csv_output, index=False)

# Send CSV to production (It could be used with other scripts)
naas.dependency.add(csv_output)